In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

print(os.getenv("AZURE_OPENAI_ENDPOINT"))

database_name = os.getenv('SQL_DATABASE_NAME')

https://openai-training-basic.openai.azure.com/


In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
)

Kernel(plugins=KernelPluginCollection(plugins={}), prompt_template_engine=PromptTemplateEngine(), memory=NullMemory(), text_completion_services={'chat_completion': <function Kernel.add_text_completion_service.<locals>.<lambda> at 0x000001E2541898A0>}, chat_services={'chat_completion': <function Kernel.add_chat_service.<locals>.<lambda> at 0x000001E254189800>}, text_embedding_generation_services={}, default_text_completion_service='chat_completion', default_chat_service='chat_completion', default_text_embedding_generation_service=None, retry_mechanism=PassThroughWithoutRetry(), function_invoking_handlers={}, function_invoked_handlers={})

In [3]:
sk_prompt = """
Generate a SQL query for a SQL Server database to retrieve data from an ERP system. Database name is called: {{$database}}
Take into consideration that the schema for the Product table is the following: {{$schema}}
The query should return the required data to satisfy the request of the user: {{$user_request}}"""

In [4]:
chat_function = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="ChatBot",
    max_tokens=2000,
    temperature=0.4,
    top_p=0.5,
)

In [7]:
schema = """ ProductID 
          int 
          PRIMARY KEY
          Not Nullable        
  
          Name 
          nvarchar 
          No Constraint
          Not Nullable        
  
          ProductNumber 
          nvarchar 
          No Constraint
          Not Nullable"""

context = kernel.create_new_context()
context["database"] = database_name
context["schema"] = schema
context["user_request"] = "I want to see what are the top 5 products in our catalog"

bot_answer = await chat_function.invoke(context=context)

print(bot_answer)


SELECT TOP 5 Name, ProductNumber
FROM AdventureWorks_v1.dbo.Product
ORDER BY ProductID DESC;
